In [1]:
pip install numpy pandas matplotlib ipython

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Завдання 1

Для кожної із адміністративних одиниць України завантажити тестові структуровані файли, що містять значення VHI-індексу. 
Ця процедура має бути автоматизована, параметром процедури має бути індекс (номер) області. 
При зберіганні файлу до його імені потрібно додати дату та час завантаження.

Передбачити повторні запуски скрипту, довантаження нових даних та колізію
даних;

In [ ]:
import urllib.request   # Модуль для роботи з HTTP-запитами
import hashlib          # Модуль для обчислення хешів (SHA-256)
import os               # Модуль для роботи з файловою системою
import re               # Модуль для регулярних виразів
from datetime import datetime  # Для формування поточної дати/часу у назві файлу

# Папка, у яку зберігатимуться CSV-файли з даними
SAVE_FOLDER = "CSV_Files"

# Якщо папка не існує — створюємо її
if not os.path.exists(SAVE_FOLDER):
    os.makedirs(SAVE_FOLDER)

def compute_sha256(content):
    """
    Обчислює SHA-256 хеш для вмісту (рядка).
    Це використовується для перевірки, чи дублюється файл.
    """
    return hashlib.sha256(content.encode('utf-8')).hexdigest()

def fetch_vhi(region_number):
    """
    Завантажує дані VHI для однієї області за її номером.
    Якщо такі дані вже існують у вигляді ідентичного файлу — не зберігає повторно.
    """
    
    # Формування URL для запиту до сервера NOAA
    url = (
        f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php"
        f"?country=UKR&provinceID={region_number}&year1=1981&year2=2024&type=Mean"
    )

    # Створення унікального імені файлу на основі поточного часу
    timestamp = datetime.now().strftime("%d-%m-%Y_%H-%M-%S")
    filename = f"region_{region_number}_VHI_{timestamp}.csv"
    filepath = os.path.join(SAVE_FOLDER, filename)

    try:
        # Відправлення запиту та отримання HTML-відповіді від сервера
        response = urllib.request.urlopen(url)
        raw_html = response.read().decode('utf-8', errors='replace')

        # Пошук CSV-даних усередині HTML-тегів <pre>...</pre>
        match = re.search(r"<pre>(.*?)</pre>", raw_html, re.DOTALL)
        if not match:
            print(f"Дані не знайдено для області {region_number}")
            return

        # Вилучення CSV-даних з тегу
        csv_data = match.group(1).strip()

        # Обчислення хешу нових CSV-даних
        new_hash = compute_sha256(csv_data)

        # Перевірка на дублікати: порівнюємо хеші з уже наявними файлами
        for old_file in os.listdir(SAVE_FOLDER):
            if not old_file.endswith('.csv'):
                continue  # Пропускаємо не-CSV файли
            old_path = os.path.join(SAVE_FOLDER, old_file)
            with open(old_path, 'r', encoding='utf-8') as f:
                old_data = f.read()
                if compute_sha256(old_data) == new_hash:
                    print(f"Дублікат знайдено для області {region_number}. Файл не буде збережено.")
                    return  # Не зберігати дублікати

        # Якщо дубліката не знайдено — зберігаємо CSV-файл
        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(csv_data)

        print(f"Файл збережено: {filename}")

    except Exception as e:
        # Вивід повідомлення, якщо щось пішло не так
        print(f"Помилка при завантаженні для області {region_number}: {e}")

# Завантаження VHI-даних для всіх 27 областей України (індекси від 1 до 27 включно)
for region_id in range(1, 28):
    fetch_vhi(region_id)


Файл збережено: region_1_VHI_11-05-2025_16-33-07.csv
Файл збережено: region_2_VHI_11-05-2025_16-33-09.csv
Файл збережено: region_3_VHI_11-05-2025_16-33-10.csv
Файл збережено: region_4_VHI_11-05-2025_16-33-14.csv
Файл збережено: region_5_VHI_11-05-2025_16-33-15.csv
Файл збережено: region_6_VHI_11-05-2025_16-33-16.csv
Файл збережено: region_7_VHI_11-05-2025_16-33-17.csv
Файл збережено: region_8_VHI_11-05-2025_16-33-18.csv
Файл збережено: region_9_VHI_11-05-2025_16-33-19.csv
Файл збережено: region_10_VHI_11-05-2025_16-33-20.csv
Файл збережено: region_11_VHI_11-05-2025_16-33-21.csv
Файл збережено: region_12_VHI_11-05-2025_16-33-22.csv
Файл збережено: region_13_VHI_11-05-2025_16-33-24.csv
Файл збережено: region_14_VHI_11-05-2025_16-33-25.csv
Файл збережено: region_15_VHI_11-05-2025_16-33-26.csv
Файл збережено: region_16_VHI_11-05-2025_16-33-27.csv
Файл збережено: region_17_VHI_11-05-2025_16-33-28.csv
Файл збережено: region_18_VHI_11-05-2025_16-33-29.csv
Файл збережено: region_19_VHI_11-05-2

In [ ]:
import pandas as pd
import os

# Шлях до теки з CSV-файлами
folder_path = 'CSV_Files'  # заміни на свій шлях

# Порожній список для збору всіх DataFrame
all_data = []

# Перебір усіх CSV у теці
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)

        # Витягуємо назву регіону з імені файлу (без .csv)
        region_name = os.path.splitext(filename)[0]

        # Додаємо колонку з назвою області
        df['region'] = region_name

        # Додаємо у загальний список
        all_data.append(df)

# Об'єднуємо всі таблиці в одну
combined_df = pd.concat(all_data, ignore_index=True)

# Зберігаємо об'єднаний файл
combined_df.to_csv('all_regions_combined.csv', index=False)

print("✅ Успішно об'єднано! Збережено як all_regions_combined.csv")


# Завдання 2

Зчитати завантажені текстові файли у фрейм
(https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) (детальніше
про роботу із фреймами буде розказано у подальших лабораторних роботах).
Імена стовбців фрейму мають бути змістовними та легкими для сприйняття (не
повинно бути спеціалізованих символів, пробілів тощо). Ця задача має бути
реалізована у вигляді окремої процедури, яка на вхід приймає шлях до
директорії, в якій зберігаються файли;

In [ ]:
import pandas as pd
import os
import re

# Папка з CSV-файлами
DATA_DIR = "CSV_Files"
RESULT_FILE = "full.csv"

# Очікувані назви колонок
COLUMNS = ["Year", "Week", "SMN", "SMT", "VCI", "TCI", "VHI", "PROVINCE_ID"]

# Ініціалізуємо фінальний DataFrame
result_df = pd.DataFrame(columns=COLUMNS)

# Отримуємо список лише тих CSV-файлів, які містять числа (ідентифікатори)
raw_files = os.listdir(DATA_DIR)
target_files = []
for file in raw_files:
    if file.endswith(".csv") and re.search(r'\d+', file):
        target_files.append(file)

# Сортуємо список файлів за першим числом (ідентифікатор області)
sorted_files = sorted(target_files, key=lambda name: int(re.findall(r'\d+', name)[0]))

# Обробляємо кожен файл
for file in sorted_files:
    file_path = os.path.join(DATA_DIR, file)

    try:
        # Витягуємо ідентифікатор області
        match = re.findall(r'\d+', file)
        province = int(match[0]) if match else None
        if province is None:
            continue

        # Зчитуємо дані
        data = pd.read_csv(file_path, skiprows=0, names=COLUMNS)

        # Очищаємо колонку 'Year' від HTML-тегів
        data["Year"] = data["Year"].astype(str).str.replace(r"<tt><pre>|</pre></tt>", "", regex=True)
        data["Year"] = pd.to_numeric(data["Year"], errors="coerce")
        data["Week"] = pd.to_numeric(data["Week"], errors="coerce")

        # Видаляємо рядки з некоректними значеннями
        data.dropna(subset=["Year", "Week"], inplace=True)
        data = data.astype({"Year": int, "Week": int})

        # Встановлюємо колонку PROVINCE_ID
        data["PROVINCE_ID"] = province

        # Фільтруємо негативні або відсутні значення VHI
        data = data[data["VHI"] != -1].dropna()

        # Додаємо до підсумкового DataFrame
        result_df = pd.concat([result_df, data], ignore_index=True)

    except Exception as err:
        print(f"Не вдалося обробити файл {file}: {err}")

# Збереження результату
result_df.to_csv(RESULT_FILE, index=False)

# Вивід результату
print(f"Дані об'єднано та збережено у '{RESULT_FILE}'")
print("Перші 10 рядків:")
print(result_df.head(10))
print("Останні 10 рядків:")
print(result_df.tail(10))


C:\Users\User\AppData\Local\Temp\ipykernel_8464\3985729364.py:55: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, data], ignore_index=True)


Дані об'єднано та збережено у 'full.csv'
Перші 10 рядків:
   Year Week    SMN     SMT    VCI    TCI    VHI PROVINCE_ID
0  1982    1  0.053  260.31  45.01  39.46  42.23           1
1  1982    2  0.054  262.29  46.83  31.75  39.29           1
2  1982    3  0.055  263.82  48.13  27.24  37.68           1
3  1982    4  0.053  265.33  46.09  23.91  35.00           1
4  1982    5  0.050  265.66  41.46  26.65  34.06           1
5  1982    6  0.048  266.55  36.56  29.46  33.01           1
6  1982    7  0.048  267.84  32.17  31.14  31.65           1
7  1982    8  0.050  269.30  30.30  32.50  31.40           1
8  1982    9  0.052  270.75  28.23  35.22  31.73           1
9  1982   10  0.056  272.73  25.25  37.63  31.44           1
Останні 10 рядків:
       Year Week    SMN     SMT    VCI    TCI    VHI PROVINCE_ID
59012  2024   43  0.259  281.45  79.71  17.45  48.58          27
59013  2024   44  0.229  279.41  76.74  13.33  45.04          27
59014  2024   45  0.206  278.07  77.64   8.70  43.17     

# Завдання 3

Реалізувати окрему процедуру, яка змінить індекси областей, які використані на
порталі NOAA (за англійською абеткою) на наступні, за українською (виключно
старі індекси на нові):

In [ ]:
import pandas as pd

def update_province_ids(df):

    # Мапінг старих індексів (NOAA) на нові (українська абетка)
    province_mapping = {
        1: 21, 2: 24, 3: 23, 4: 26, 5: 3, 6: 4, 7: 8, 8: 19, 9: 20, 10: 22,
        11: 9, 12: 10, 13: 11, 14: 12, 15: 13, 16: 14, 17: 15, 18: 16, 19: 17,
        20: 18, 21: 1, 22: 2, 23: 6, 24: 7, 25: 5, 26: 25, 27: 27

        }
    
    # Переконуємося, що колонка "PROVINCE_ID" є у DataFrame
    if "PROVINCE_ID" not in df.columns:
        print("Колонка 'PROVINCE_ID' не знайдена у DataFrame!")
        return df

    # Оновлення значень
    df["PROVINCE_ID"] = df["PROVINCE_ID"].map(province_mapping)

    # Приведення стовпця "Week" до числового типу та обробка некоректних значень
    if 'Week' in df.columns:
        df['Week'] = pd.to_numeric(df['Week'], errors='coerce')

    return df

# Припустимо, що дані зчитуються з файлу CSV
input_file = "full.csv"
output_file = "Updated_Provinces.csv"

# Зчитуємо дані з файлу
combined_data = pd.read_csv(input_file)

# Оновлення індексів у `combined_data`
combined_data = update_province_ids(combined_data)

# Збереження у файл
combined_data.to_csv(output_file, index=False)

# Виведення результату
print(f"Оновлений файл збережено: {output_file}")
print("Перші 10 рядків оновленого файлу:")
print(combined_data.head(10))

Оновлений файл збережено: Updated_Provinces.csv
Перші 10 рядків оновленого файлу:
   Year  Week    SMN     SMT    VCI    TCI    VHI  PROVINCE_ID
0  1982     1  0.053  260.31  45.01  39.46  42.23           21
1  1982     2  0.054  262.29  46.83  31.75  39.29           21
2  1982     3  0.055  263.82  48.13  27.24  37.68           21
3  1982     4  0.053  265.33  46.09  23.91  35.00           21
4  1982     5  0.050  265.66  41.46  26.65  34.06           21
5  1982     6  0.048  266.55  36.56  29.46  33.01           21
6  1982     7  0.048  267.84  32.17  31.14  31.65           21
7  1982     8  0.050  269.30  30.30  32.50  31.40           21
8  1982     9  0.052  270.75  28.23  35.22  31.73           21
9  1982    10  0.056  272.73  25.25  37.63  31.44           21


# Завдання 4

### Реалізувати процедури для формування вибірок наступного виду (включаючи елементи аналізу):

o Ряд VHI для області за вказаний рік;

1. Отримати ряд VHI для області за вказаний рік

In [34]:
def get_vhi_for_region(df, province_id, year=None):

    # Перевірка наявності колонок
    required_columns = ["PROVINCE_ID", "Year", "Week", "VHI"]
    missing_cols = [col for col in required_columns if col not in df.columns]
    if missing_cols:
        print(f"Попередження: У DataFrame відсутні колонки {missing_cols}")
        return pd.DataFrame(columns=required_columns)
    
    # Фільтрація даних
    mask = df["PROVINCE_ID"] == province_id
    if year is not None:
        mask &= df["Year"] == year
    
    result = df.loc[mask, ["Year", "Week", "VHI"]]
    
    # Сортування за роком та тижнем (якщо дані є)
    if not result.empty:
        result = result.sort_values(["Year", "Week"])
    
    return result

# Приклад використання
vhi_data = get_vhi_for_region(combined_data, province_id=21, year=1997)
print("Дані VHI для Вінницької області (ID=21) у 1997 році:")
print(vhi_data)

Дані VHI для Вінницької області (ID=21) у 1997 році:
     Year  Week    VHI
750  1997     1  30.59
751  1997     2  31.96
752  1997     3  33.30
753  1997     4  33.85
754  1997     5  36.83
755  1997     6  39.52
756  1997     7  38.44
757  1997     8  36.73
758  1997     9  36.21
759  1997    10  35.20
760  1997    11  32.23
761  1997    12  30.84
762  1997    13  31.11
763  1997    14  30.91
764  1997    15  31.23
765  1997    16  33.55
766  1997    17  37.09
767  1997    18  42.06
768  1997    19  46.30
769  1997    20  47.73
770  1997    21  51.82
771  1997    22  57.77
772  1997    23  62.80
773  1997    24  68.77
774  1997    25  73.78
775  1997    26  76.53
776  1997    27  79.05
777  1997    28  81.06
778  1997    29  81.70
779  1997    30  82.47
780  1997    31  82.28
781  1997    32  82.45
782  1997    33  82.96
783  1997    34  83.20
784  1997    35  83.70
785  1997    36  83.44
786  1997    37  82.34
787  1997    38  80.51
788  1997    39  77.40
789  1997    40  73.30
790 

2. Пошук екстремумів (min, max), середнього, медіани

In [36]:
def get_vhi_statistics(df, province_ids, years=None):

    # Перевірка наявності колонок
    required_columns = ["PROVINCE_ID", "Year", "VHI"]
    missing_cols = [col for col in required_columns if col not in df.columns]
    if missing_cols:
        print(f"Попередження: У DataFrame відсутні колонки {missing_cols}")
        return pd.DataFrame(columns=required_columns + ["min", "max", "mean", "median"])
    
    # Фільтрація даних
    mask = df["PROVINCE_ID"].isin(province_ids)
    if years is not None:
        mask &= df["Year"].isin(years)
    
    filtered_df = df.loc[mask]
    
    # Обчислення статистики
    if filtered_df.empty:
        print("Попередження: Немає даних для вказаних параметрів")
        return pd.DataFrame(columns=required_columns + ["min", "max", "mean", "median"])
    
    stats = filtered_df.groupby(["PROVINCE_ID", "Year"])["VHI"] \
                      .agg(["min", "max", "mean", "median"]) \
                      .reset_index()
    
    return stats

# Приклад використання
vhi_stats = get_vhi_statistics(
    combined_data, 
    province_ids=[2, 27],  
    years=[1997, 2007]     
)

print("Статистика VHI:")
print(vhi_stats)


Статистика VHI:
   PROVINCE_ID  Year    min    max       mean  median
0            2  1997  32.29  75.08  53.080192  50.525
1            2  2007  35.04  57.10  48.585769  49.945
2           27  1997  44.41  75.39  57.640000  54.540
3           27  2007  39.81  62.78  50.675962  50.745


3. Отримати ряд VHI за вказаний діапазон років для вказаних областей

In [ ]:
def get_vhi_by_year_range(df, province_ids, start_year, end_year, include_stats=False, verbose=True):

    # Перевірка наявності необхідних колонок
    required_cols = ['PROVINCE_ID', 'Year', 'Week', 'VHI']
    missing_cols = [col for col in required_cols if col not in df.columns]
    
    if missing_cols:
        msg = f"Попередження: Відсутні колонки: {missing_cols}"
        if verbose: print(msg)
        return pd.DataFrame(columns=required_cols)
    
    # Перевірка коректності років
    if start_year > end_year:
        msg = "Попередження: Початковий рік більший за кінцевий. Буде поміняно місцями."
        if verbose: print(msg)
        start_year, end_year = end_year, start_year
    
    # Перевірка наявності вказаних областей
    existing_provinces = df['PROVINCE_ID'].unique()
    missing_provinces = [pid for pid in province_ids if pid not in existing_provinces]
    
    if missing_provinces and verbose:
        print(f"Попередження: Відсутні дані для областей з ID: {missing_provinces}")
    
    # Фільтрація даних
    mask = (df['PROVINCE_ID'].isin(province_ids)) & (df['Year'].between(start_year, end_year))
    result = df.loc[mask, required_cols]
    
    if result.empty:
        msg = f"Попередження: Немає даних для областей {province_ids} у період {start_year}-{end_year}"
        if verbose: print(msg)
        return pd.DataFrame(columns=required_cols)
    
    # Сортування результатів
    result = result.sort_values(['PROVINCE_ID', 'Year', 'Week'])
    
    # Додаткова інформація про наявні області
    if verbose:
        present_provinces = result['PROVINCE_ID'].unique()
        print(f"Знайдено дані для областей з ID: {sorted(present_provinces)}")
        print(f"Діапазон років: {result['Year'].min()} - {result['Year'].max()}")
        print(f"Кількість записів: {len(result)}")
    
    # Додаткова статистика
    if include_stats:
        stats = result.groupby(['PROVINCE_ID', 'Year'])['VHI'] \
                     .agg(['min', 'max', 'mean', 'median']) \
                     .reset_index()
        return {'data': result, 'stats': stats}
    
    return result

# Приклад використання:
province_ids = [3, 7, 12] 
start_year = 1990
end_year = 2000

print("Отримання даних...")
vhi_data = get_vhi_by_year_range(combined_data, province_ids, start_year, end_year, verbose=True)

if not vhi_data.empty:
    print("\nПерші 10 рядків даних:")
    print(vhi_data.head(10))
    
    # Виводимо дані для кожної області окремо
    for province in province_ids:
        province_data = vhi_data[vhi_data['PROVINCE_ID'] == province]
        if not province_data.empty:
            print(f"\nДані для області ID={province}:")
            print(province_data.head())
        else:
            print(f"\nДля області ID={province} дані відсутні")
else:
    print("Дані не знайдено")

Отримання даних...
Знайдено дані для областей з ID: [np.int64(3), np.int64(7), np.int64(12)]
Діапазон років: 1990 - 2020
Кількість записів: 4719

Перші 10 рядків даних:
      PROVINCE_ID  Year  Week    VHI
9149            3  1990     1  40.88
9150            3  1990     2  41.08
9151            3  1990     3  41.28
9152            3  1990     4  41.70
9153            3  1990     5  41.75
9154            3  1990     6  42.43
9155            3  1990     7  44.36
9156            3  1990     8  45.45
9157            3  1990     9  46.07
9158            3  1990    10  51.72

Дані для області ID=3:
      PROVINCE_ID  Year  Week    VHI
9149            3  1990     1  40.88
9150            3  1990     2  41.08
9151            3  1990     3  41.28
9152            3  1990     4  41.70
9153            3  1990     5  41.75

Дані для області ID=7:
       PROVINCE_ID  Year  Week    VHI
50683            7  1990     1  39.52
50684            7  1990     2  40.34
50685            7  1990     3  41.14
50

o Для всього набору даних виявити роки, протягом яких екстремальні
посухи торкнулися більше вказаного відсотка областей по Україні (20%
областей - 5 областей з 25). Повернути роки, назви областей з
екстремальними посухами та значення VHI;

4. Виявити роки, коли посуха торкнулася >20% областей (VHI < 15)

In [49]:
def get_extreme_drought_data(df, threshold=15, min_provinces=5, start_year=None, end_year=None, verbose=True):

    # Перевірка колонок
    required_cols = ['PROVINCE_ID', 'Year', 'VHI']
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        print(f"Відсутні колонки: {missing_cols}")
        return pd.DataFrame(columns=required_cols)
    
    # Фільтрація за роками
    if start_year and end_year:
        if start_year > end_year:
            start_year, end_year = end_year, start_year
        df = df[df['Year'].between(start_year, end_year)]
    
    # Знаходимо записи з екстремальною посухою
    drought_data = df[df['VHI'] < threshold]
    
    if drought_data.empty:
        print(f"Не знайдено записів з VHI < {threshold}")
        return pd.DataFrame(columns=required_cols)
    
    # Визначаємо роки з достатньою кількістю провінцій
    yearly_counts = drought_data.groupby('Year')['PROVINCE_ID'].nunique()
    extreme_years = yearly_counts[yearly_counts >= min_provinces].index
    
    if not extreme_years.any():
        print(f"Не знайдено років з ≥{min_provinces} провінціями (VHI < {threshold})")
        return pd.DataFrame(columns=required_cols)
    
    # Фільтруємо дані для цих років
    result = drought_data[drought_data['Year'].isin(extreme_years)]
    
    if verbose:
        print(f"Знайдено {len(extreme_years)} років з ≥{min_provinces} провінціями (VHI < {threshold}):")
        print(sorted(extreme_years))
        print("\nПерші 10 записів:")
        print(result[['Year', 'PROVINCE_ID', 'VHI']].head(10))
    
    return result[['Year', 'PROVINCE_ID', 'VHI']].sort_values(['Year', 'PROVINCE_ID'])

# Приклад використання
print("Отримання даних про екстремальну посуху...")
drought_table = get_extreme_drought_data(
    combined_data,
    threshold=15,
    min_provinces=5,
    start_year=2000,
    end_year=2020
)

if not drought_table.empty:
    print("\nПовна таблиця даних:")
    print(drought_table)
    
    # Додаткова статистика
    print("\nСтатистика по роках:")
    stats = drought_table.groupby('Year')['PROVINCE_ID'].agg(['count', 'nunique'])
    stats.columns = ['Записи', 'Унікальні провінції']
    print(stats)
else:
    print("Дані не знайдено")

Отримання даних про екстремальну посуху...
Знайдено 2 років з ≥5 провінціями (VHI < 15):
[2000, 2007]

Перші 10 записів:
      Year  PROVINCE_ID    VHI
949   2000           21  14.64
950   2000           21  11.82
951   2000           21  10.81
952   2000           21  10.68
953   2000           21  12.30
954   2000           21  14.24
7837  2007           26  14.98
7838  2007           26  14.23
7839  2007           26  13.79
7840  2007           26  13.41

Повна таблиця даних:
       Year  PROVINCE_ID    VHI
51228  2000            7  12.26
51229  2000            7  11.28
51230  2000            7  11.25
51231  2000            7  11.38
51232  2000            7  12.91
...     ...          ...    ...
7838   2007           26  14.23
7839   2007           26  13.79
7840   2007           26  13.41
7841   2007           26  13.28
7842   2007           26  14.36

[88 rows x 3 columns]

Статистика по роках:
      Записи  Унікальні провінції
Year                             
2000      41       